Cartar de posible desvio para la union

In [2]:
import pandas as pd
import collections

from docxtpl import DocxTemplate

In [3]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [5]:
ruta

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [6]:
path_xlsx_avance = r'G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/AVANCE DE COSECHA V2.xlsx'

In [8]:
xlsx_entregas = pd.read_excel(path_xlsx_avance, sheet_name='ENTREGAS')
xlsx_avance = pd.read_excel(path_xlsx_avance, sheet_name = 'AVANCE POR GRUPO')

In [9]:
xlsx_entregas.head(3)

,INST,GRUPO,CODIGO CAÑERO,NOMBRE CAÑERO,COMPROMISO DE ENTREGA,PAQ,TM. ENTREGADAS,AVANCE,INICIO DE ENTREGAS,ULTIMA ENTREGA,COD_COSECHA
0,131,3,17,ACSAMA SEGOVIA SABINO,565.19,30.0,541.02,0.957236,45136.0,45139.0,356
1,126,1,18,ACUNA ARISPE HONORATO,517.69,24.0,364.33,0.703761,45138.0,45163.0,354
2,59,7,39,AGUILAR ROMERO NICOLAS,1020.46,72.0,782.38,0.766693,45080.0,45169.0,32


In [10]:
xlsx_avance.head(3)

,COD_COS,COMPROMISO,ENTREGA,_ENTREGA,TOTAL_AREA,COSECHA,SIN COSECHA,%COSECHA,TCH,DIF.AVANCE,...,TN_COSECHA_ESTIM,DIF.ESTIM,% DIF.ESTIM,TN_EN_CAMPO,COMPROMISO_RESTANTE,DIFERENCIA,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,1,26449.77,20331.96,0.768701,543.38,327.87,215.51,0.603390,62.012261,0.165311,...,18177.722033,2154.237967,0.894047,11948.274851,6117.81,5830.464851,NaN,NaN,NaN,NaN
1,2,10848.49,9950.67,0.917240,170.47,121.17,49.30,0.710800,82.121565,0.206441,...,6168.160177,3782.509823,0.619874,2509.617040,897.82,1611.797040,NaN,NaN,NaN,NaN
2,3,5390.78,3570.04,0.662249,112.85,48.95,63.90,0.433762,72.932380,0.228488,...,2372.404219,1197.635781,0.664532,3096.968939,1820.74,1276.228939,NaN,NaN,NaN,NaN


## Argumentos

In [11]:
fecha_inf = '02 de septiembra de 2023'
fecha_form = '02-09-2023'

fecha_img = '30 de agosto de 2023'
cod_grupo = 30
cod_grupos = [29, 103]

In [12]:
for i in cod_grupos:
    # datos del grupo
    grupo = xlsx_entregas[xlsx_entregas['COD_COSECHA'] == i].copy()
    grupo = grupo[['INST', 'CODIGO CAÑERO ', 'NOMBRE CAÑERO','COMPROMISO DE ENTREGA', 'TM. ENTREGADAS']]
    grupo['TM. ENTREGADAS'] = grupo['TM. ENTREGADAS'].fillna(0)
    grupo['avance%'] = grupo['TM. ENTREGADAS'] / grupo['COMPROMISO DE ENTREGA'] * 100
    
    # convertir datos del grupo a objetos
    list_obj_caneros = []
    for index, row in grupo.iterrows():
        canero = {
            'inst' : row['INST'],
            'cod_ca' : row['CODIGO CAÑERO '],
            'nom_ca' : row['NOMBRE CAÑERO'],
            'compromiso' : row['COMPROMISO DE ENTREGA'],
            'entrega' : row['TM. ENTREGADAS'],
            'entrega_porcen' : row['avance%']
        }
        list_obj_caneros.append(collections.namedtuple("canero", canero.keys())(*canero.values()))
    
    # totales de los datos del grupo
    grupo_totales = {
        'compromiso' : grupo['COMPROMISO DE ENTREGA'].sum(),
        'entregas' : grupo['TM. ENTREGADAS'].sum(),
        'entrega_porcen' : grupo['TM. ENTREGADAS'].sum() / grupo['COMPROMISO DE ENTREGA'].sum() * 100
    }
    obj_grupo_totales = collections.namedtuple("grupo_totales", grupo_totales.keys())(*grupo_totales.values())
    
    # datos generales de avance de cosecha
    avance = xlsx_avance[xlsx_avance['COD_COS'] == i]
    avance_totales = {
        'fecha_img' : fecha_img,
        'fecha_informe' : fecha_inf,
        'compromiso' : float(avance['COMPROMISO']),
        'entrega' : float(avance['ENTREGA']),
        'entrega_porcen' : float(avance['_ENTREGA']) * 100,
        'ha_canha' : float(avance['TOTAL_AREA']),
        'ha_cosecha' : float(avance['COSECHA']),
        'cosecha_porcen' : float(avance['%COSECHA']) * 100,
        'tch' : float(avance['TCH'])
    }
    
    # datos generales a objeto
    obj_avance_totales = collections.namedtuple("avance_totales",avance_totales.keys())(*avance_totales.values())
    
    # generar reporte
    doc = DocxTemplate(ruta + '/templates/tpl_rpt_avance_posible_desvio.docx')
    context = {
        'insp' : list_obj_caneros,
        'grupo_totales' : obj_grupo_totales,
        'avance' : obj_avance_totales
    }
    doc.render(context)
    doc.save(ruta + '/informes/_' + str(i) + '_RDC_' + fecha_form + '.docx')
    print('Se generó reporte de grupo:', i)

Se generó reporte de grupo: 29
Se generó reporte de grupo: 103


## DATOS DEL GRUPO

In [11]:
grupo = xlsx_entregas[xlsx_entregas['COD_COSECHA'] == cod_grupo].copy()
grupo = grupo[['INST', 'CODIGO CAÑERO ', 'NOMBRE CAÑERO','COMPROMISO DE ENTREGA', 'TM. ENTREGADAS']]
grupo['TM. ENTREGADAS'] = grupo['TM. ENTREGADAS'].fillna(0)
grupo['avance%'] = grupo['TM. ENTREGADAS'] / grupo['COMPROMISO DE ENTREGA'] * 100
grupo

,INST,CODIGO CAÑERO,NOMBRE CAÑERO,COMPROMISO DE ENTREGA,TM. ENTREGADAS,avance%
6,50,75,AGUILERA TARADELLES NELSON MARIANO,26992.97,10171.47,37.681922
48,50,577,AGUILERA MARIA INGRID WENDE DE,13433.25,13646.16,101.584948
103,50,794,AGUILERA WENDE MARIANO,18084.70,18219.59,100.745879
561,50,8005,AGUILERA WENDE MARIA INGRID,5201.02,54.64,1.050563
1151,50,41715,VACA DIEZ UGARTE NICOLAS,1781.38,0.00,0.000000


In [12]:
list_obj_caneros = []
for index, row in grupo.iterrows():
    canero = {
        'inst' : row['INST'],
        'cod_ca' : row['CODIGO CAÑERO '],
        'nom_ca' : row['NOMBRE CAÑERO'],
        'compromiso' : row['COMPROMISO DE ENTREGA'],
        'entrega' : row['TM. ENTREGADAS'],
        'entrega_porcen' : row['avance%']
    }
    list_obj_caneros.append(collections.namedtuple("canero", canero.keys())(*canero.values()))
list_obj_caneros

[canero(inst=50, cod_ca=75, nom_ca='AGUILERA TARADELLES NELSON MARIANO ', compromiso=26992.97, entrega=10171.47, entrega_porcen=37.68192236719412),
 canero(inst=50, cod_ca=577, nom_ca='AGUILERA MARIA INGRID WENDE DE', compromiso=13433.25, entrega=13646.16, entrega_porcen=101.58494779744291),
 canero(inst=50, cod_ca=794, nom_ca='AGUILERA WENDE MARIANO', compromiso=18084.7, entrega=18219.59, entrega_porcen=100.7458791132836),
 canero(inst=50, cod_ca=8005, nom_ca='AGUILERA WENDE MARIA INGRID ', compromiso=5201.02, entrega=54.64, entrega_porcen=1.0505631587650115),
 canero(inst=50, cod_ca=41715, nom_ca='VACA DIEZ UGARTE NICOLAS', compromiso=1781.38, entrega=0.0, entrega_porcen=0.0)]

In [13]:
grupo_totales = {
    'compromiso' : grupo['COMPROMISO DE ENTREGA'].sum(),
    'entregas' : grupo['TM. ENTREGADAS'].sum(),
    'entrega_porcen' : grupo['TM. ENTREGADAS'].sum() / grupo['COMPROMISO DE ENTREGA'].sum() * 100
}
obj_grupo_totales = collections.namedtuple("grupo_totales", grupo_totales.keys())(*grupo_totales.values())
obj_grupo_totales

grupo_totales(compromiso=65493.32, entregas=42091.86, entrega_porcen=64.26893612966941)

## DATOS DE AVANCE

In [14]:
xlsx_avance.head(3)

,COD_COS,COMPROMISO,ENTREGA,_ENTREGA,TOTAL_AREA,COSECHA,SIN COSECHA,%COSECHA,TCH,DIF.AVANCE,AREA_ESTIM.,TCH_ESTIM,PRODUCCION_ESTIM,TN_COSECHA_ESTIM,DIF.ESTIM,% DIF.ESTIM,TN_EN_CAMPO,COMPROMISO_RESTANTE,DIFERENCIA
0,1,26449.77,19161.65,0.724454,543.38,304.38,239.00,0.560160,62.953052,0.164294,543.960674,55.385149,30127.342884,16876.146761,2285.503239,0.880725,13251.196123,7288.12,5963.076123
1,2,10848.49,5689.97,0.524494,170.45,60.19,110.26,0.353124,94.533477,0.171370,170.249063,50.971072,8677.777217,3064.332125,2625.637875,0.538550,5613.445092,5158.52,454.925092
2,3,5390.78,2792.22,0.517962,112.86,40.34,72.52,0.357434,69.217154,0.160528,114.920802,47.592543,5469.373160,1954.939866,837.280134,0.700138,3514.433294,2598.56,915.873294


In [15]:
avance = xlsx_avance[xlsx_avance['COD_COS'] == cod_grupo]
avance

,COD_COS,COMPROMISO,ENTREGA,_ENTREGA,TOTAL_AREA,COSECHA,SIN COSECHA,%COSECHA,TCH,DIF.AVANCE,AREA_ESTIM.,TCH_ESTIM,PRODUCCION_ESTIM,TN_COSECHA_ESTIM,DIF.ESTIM,% DIF.ESTIM,TN_EN_CAMPO,COMPROMISO_RESTANTE,DIFERENCIA
28,30,65493.32,42091.86,0.642689,856.71,452.11,404.6,0.527728,93.100927,0.114961,857.018376,76.385829,65464.059007,34547.228021,7544.631979,0.820758,30916.830986,23401.46,7515.370986


In [29]:
avance_totales = {
    'fecha_img' : fecha_img,
    'fecha_informe' : fecha_inf,
    'compromiso' : float(avance['COMPROMISO']),
    'entrega' : float(avance['ENTREGA']),
    'entrega_porcen' : float(avance['_ENTREGA']) * 100,
    'ha_canha' : float(avance['TOTAL_AREA']),
    'ha_cosecha' : float(avance['COSECHA']),
    'cosecha_porcen' : float(avance['%COSECHA']) * 100,
    'tch' : float(avance['TCH'])
}
avance_totales

{'fecha_img': '05 de agosto de 2023',
 'fecha_informe': '14 de agosto de 2023',
 'compromiso': 65493.32,
 'entrega': 42091.86,
 'entrega_porcen': 64.26893612966941,
 'ha_canha': 856.71,
 'ha_cosecha': 452.11,
 'cosecha_porcen': 52.77281693922097,
 'tch': 93.10092676561014}

In [67]:
obj_avance_totales = collections.namedtuple("avance_totales",avance_totales.keys())(*avance_totales.values())
obj_avance_totales

avance_totales(fecha_img='05 de agosto de 2023', fecha_informe='14 de agosto de 2023', compromiso=26449.77, entrega=19161.65, entrega_porcen=72.44543147256101, ha_canha=543.38, ha_cosecha=304.38, cosecha_porcen=56.01604770142442, tch=62.953052105920236)

In [68]:
doc = DocxTemplate(ruta + '/templates/tbl_rpt_posible_desvio.docx')
context = {
    'insp' : list_obj_caneros,
    'grupo_totales' : obj_grupo_totales,
    'avance' : obj_avance_totales
}
doc.render(context)
doc.save(ruta + '/informes/rpt.docx')